In [27]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from Model import UNet
from torchsummary import summary
from DatasetClass import BrainTumorSegmentationDataset
from pathlib import Path
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm import tqdm
from DiceLoss import DiceLoss

In [28]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cpu")
device

device(type='cpu')

In [ ]:
def preprocessing(root_dir, train_test_ratio=0.8, train_val_ratio=0.25, target_size=(572, 572), random_state=42):

    image_dir = Path(root_dir) / "images"
    mask_dir = Path(root_dir) / "masks"

    image_paths = list(image_dir.glob("*.png"))
    mask_paths = list(mask_dir.glob("*.png"))

    train_image_paths, test_image_paths, train_mask_paths, test_mask_paths = train_test_split(
        image_paths, mask_paths, train_size=train_test_ratio, random_state=random_state
    )
    
    train_image_paths, val_image_paths, train_mask_paths, val_mask_paths = train_test_split(
        train_image_paths, train_mask_paths, train_size=train_val_ratio, random_state=random_state
    )

    data_transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor(),
    ])

    train_dataset = BrainTumorSegmentationDataset(
        train_image_paths, 
        train_mask_paths,
        transform=data_transform,
    )
    val_dataset = BrainTumorSegmentationDataset(
        val_image_paths, 
        val_mask_paths,
        transform=data_transform,
    )

    test_dataset = BrainTumorSegmentationDataset(
        test_image_paths, 
        test_mask_paths,
        transform=data_transform,
    )

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

    return train_loader, val_loader, test_loader

root_dir = "segmentation_dataset"
train_loader, val_loader, test_loader = preprocessing(root_dir=root_dir)

In [30]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader, device):
    train_losses = []
    val_losses = []
    
    for epoch in tqdm(range(n_epochs)):

        model.train()
        train_loss = 0
        for idx, img_mask in enumerate(tqdm(train_loader)):
            img = img_mask[0].float().to(device)
            mask = img_mask[1].float().to(device)

            y_pred = model(img)
            optimizer.zero_grad()

            loss = loss_fn(y_pred, mask)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

        train_loss = train_loss / idx+1
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for idx, img_mask in enumerate(tqdm(val_loader)):
                img = img_mask[0].float().to(device)
                mask = img_mask[1].float().to(device)
                y_pred = model(img)
                loss = loss_fn(y_pred, mask)
                val_loss += loss.item()
            
            val_loss = val_loss / idx+1
            val_losses.append(val_loss)
        
        print("-"*30)
        print(f"Epoch: {epoch+1}, Train Loss: {train_loss:.4f}")
        print(f"Epoch: {epoch+1}, Val Loss: {val_loss:.4f}")
    
    return train_losses, val_losses     

In [31]:
n_epochs = 100
learning_rate = 1e-4
model = UNet(in_channels=1, out_channels=2).to(device)
loss_fn = DiceLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_losses, val_losses = training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader, device)

  0%|          | 0/100 [00:06<?, ?it/s]


RuntimeError: The size of tensor a (209952) must match the size of tensor b (262144) at non-singleton dimension 1